In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [14]:
import pandas as pd
import glob

In [17]:
def show(table):
    with pd.option_context('display.max_colwidth', None):
        display(table)

In [22]:
path = "plpr-scraper/data/out"
all_files = glob.glob(path + "/*.csv")

df_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, escapechar="\\")
    df_list.append(df)
    
df = pd.concat(df_list, axis=0, ignore_index=True)
df.text = df.text.str.replace('\n', ' ')
show(df[:50])

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party
0,2490136,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,0,"Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. - Liebe Kolleginnen und Kollegen, ich begrüße Sie zu unserer Plenarsitzung. Die Kollegin Dr. Lukrezia Jochimsen hat am 1. März ihren 75. Geburtstag gefeiert und die Kollegin Edelgard Bulmahn einige Tage später ihren 60. Geburtstag. Im Namen des gesamten Hauses möchte ich dazu auch auf diesem Wege noch einmal herzlich gratulieren und alle guten Wünsche übermitteln.",data/txt/17096.txt,chair,NaN
1,2490137,96,17,0,NaN,NaN,NaN,1,Beifall,data/txt/17096.txt,poi,NaN
2,2490138,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,2,Der Kollege Holger Haibach hat mit Wirkung vom 1. März auf seine Mitgliedschaft im Deutschen Bundestag verzichtet. Als seinen Nachfolger begrüße ich den Kollegen Helmut Heiderich.,data/txt/17096.txt,chair,NaN
3,2490139,96,17,0,NaN,NaN,NaN,3,Beifall,data/txt/17096.txt,poi,NaN
4,2490140,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,4,"Ebenso herzlich willkommen heiße ich den Kollegen Ingo Egloff, der als Nachfolger des Kollegen Olaf Scholz die Mitgliedschaft im Deutschen Bundestag erworben hat.",data/txt/17096.txt,chair,NaN
5,2490141,96,17,0,NaN,NaN,NaN,5,Beifall,data/txt/17096.txt,poi,NaN
6,2490142,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,6,"Die CDU/CSU-Fraktion hat mitgeteilt, dass die Kollegin Sibylle Pfeiffer ihr Amt als Schriftführerin niedergelegt hat.",data/txt/17096.txt,chair,NaN
7,2490143,96,17,0,Stefan Müller [Erlangen] [CDU/CSU],Stefan Müller,stefan-muller,7,Dem kann nicht zugestimmt werden!,data/txt/17096.txt,poi,cducsu
8,2490144,96,17,0,Sören Bartol [SPD],Sören Bartol,soren-bartol,8,Wollte sie keine Krawatte tragen?,data/txt/17096.txt,poi,spd
9,2490145,96,17,0,Dr. Dagmar Enkelmann [DIE LINKE],Dr. Dagmar Enkelmann,dagmar-enkelmann,9,Weil sie keine Krawatte trägt?,data/txt/17096.txt,poi,linke
